# Agent-based credit risk modeling

Summer School Bayonne July 2024<br>
Author:      Dr. Mario Gellrich<br><br>
Last update: 2024-07-01

<p>The term 'credit risk' refers to the risk that a lender may not receive the owed principal and interest. The principal is the amount a consumer borrowed and have to pay back. Interest is what the lender charges for lending the money. The credit risk of a borrower can be measured by the five Cs (see: <a>https://www.openriskmanual.org/wiki/Five_Cs_Of_Credit_Analysis</a>): capacity, capital, character, colletaral, conditions. Based on this information a credit score can be calculated for every consumer. A credit score is one indicator that lenders use to asses how likely it is that a borrower is to default. To compensate for the credit risk, consumers with lower credit scores are usually charged higher interest rates on loans than consumers with higher credit scores.</p>

<p>The agent-based model (ABM) provided in this Jupyter notebook is used to explore the role of different parameters like the number of agents and loan term on the credit risk. It contains a Lender class, a Borrower class and a CreditModel class. The model contains different time steps corresponding to the number of month in which the loan must be repayed (the loan term). The model also contains different borrower agents. Each borrower has a credit score ranging from 50 to 100. The interest rate depends on the credit risk score. If the credit risk score is high, the interest rate is low and vice versa.</p>
    
<p>At the beginning, each borrower asks for a random amount of money (the principle) between 50,000 and 500,000 USD. The model steps correspond to the number of months in which the loan must be repayed. At each model step, the borrower must repay the monthly loan. As in practise, the monthly payment remains the same throughout the loan term, but the allocation between principal and interest changes over time. At the beginning, a larger portion of the payment goes towards interest, while towards the end, a larger portion goes towards the principal. Sometimes a borrower cannot repay the monthly loan. If this happens, the borrower must repay the monthly rate owed in the next model step. If a borrower cannot repay the loan for three consecutive months, this borrower is removed from the model, and the remaining loan is considered as defaulted.</p>

## Model description

## Libraries and settings

In [46]:
# Libraries
import os
import json
import openai
import random
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from mesa import Agent, Model
from mesa.time import RandomActivation

# Set random seed
random.seed(42)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# OpenAI API key
credentials = json.loads(Path('credentials.json').read_text())['openai']
openai.api_key = credentials.get('api_key')

# Show current working directory
print(os.getcwd())

/workspaces/Summerschool_FS2024


## Example loan calculation

See: https://www.calculator.net/loan-calculator.html

In [47]:
# Input
# p = loan amount (principle)
# r = monthly interest rate
# y = number of years
# n = total number of months

p = 100000
r = 0.05 / 12
y = 30
n = y*12

# Monthly loan
m = p * r * (1 + r)**n / ((1 + r)**n - 1)

# Summary of results
print(f"Principle: {p:.0f} USD")
print(f"Loan term: {y} years ({y*12} months)")
print(f"Interest rate: {r*12*100:.2f} %")
print(f"Monthly loan: {m:.2f} USD")
print(f"Annually loan: {m*12:.2f} USD")
print(f"Total interest over {y} years: {m*n - p:.2f} USD")
print(f"Principle plus interest over {y} years: {m*n:.2f} USD")

Principle: 100000 USD
Loan term: 30 years (360 months)
Interest rate: 5.00 %
Monthly loan: 536.82 USD
Annually loan: 6441.86 USD
Total interest over 30 years: 93255.78 USD
Principle plus interest over 30 years: 193255.78 USD


## Let gpt-3.5-turbo create a function to calculate the monthly loan

In [48]:
# Function to generate a function code using GPT-3.5 Turbo
def generate_function_code(description):
    prompt = f"Write a Python function that does the following: {description}"
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message['content'].strip()

# Example description for the function
description = """Calculate the monthly loan payment for a given loan amount, 
                 interest rate, and loan term in years. Test the function with
                 a loan amount of $100,000, an interest rate of 5%, and a loan term
                 of 30 years."""

# Generate the function code
function_code = generate_function_code(description)
print("Generated function code:")
print(function_code)

Generated function code:
Here's a Python function that calculates the monthly loan payment based on the loan amount, interest rate, and loan term provided:
```python
def calculate_monthly_payment(loan_amount, interest_rate, loan_term):
    monthly_interest_rate = interest_rate / 100 / 12
    total_payments = loan_term * 12
    monthly_payment = (loan_amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -total_payments)
    return monthly_payment

loan_amount = 100000
interest_rate = 5
loan_term = 30

monthly_payment = calculate_monthly_payment(loan_amount, interest_rate, loan_term)
print(f"The monthly loan payment for a $100,000 loan at 5% interest rate for 30 years is: ${monthly_payment:.2f}")
```

Run this code in a Python environment to calculate and display the monthly loan payment for the given loan amount, interest rate, and loan term.


## Basic agent-based credit risk model

### Lender class

In [49]:
class Lender_(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    # Calculate interest rate depending on the borrowers credit score
    def calculate_interest_rate(self, borrower):
        interest_rate = 0.04 + ((100 - borrower.credit_score) * 0.0004)
        
        return interest_rate

### Borrower class

In [50]:
class Borrower_(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.credit_score = random.randint(50, 100)
        self.loan_amount = 100
        self.principle = self.loan_amount
        self.first_zero_balance_step = None  # attribute to track the step when the balance reaches zero
    
    # Display balance
    def display_balance(self):
        print(f"Borrower {self.unique_id + 1}: Balance at step 0: {self.loan_amount:.2f}.- USD")

    # Loan payments per step (in this example, only the interest is payed)
    def step(self):
        repayment_amount = self.principle * self.interest_rate
        self.loan_amount -= repayment_amount

### CreditModel class

In [51]:
class CreditModel_(Model):
    def __init__(self, num_borrowers):
        self.num_agents = num_borrowers
        self.schedule = RandomActivation(self)
        self.step_number = 0

        for i in range(self.num_agents):
            a = Borrower_(i, self)
            self.schedule.add(a)

        # Instance of Lender class
        self.lender = Lender_(0, self)

        # Calculate and show interest rates of borrower agents
        self.interest_rate()

    def interest_rate(self):
        borrowers = self.schedule.agents
        for borrower in borrowers:
            borrower.interest_rate = self.lender.calculate_interest_rate(borrower)
            print(f"Interest rate of Borrower {borrower.unique_id + 1}: {borrower.interest_rate:.2%}")

    def step(self):
        self.step_number += 1
        self.schedule.step()


### Calling the Borrower, Lender and CreditModel classes

In [56]:
# Create a model with three agents
print("Interest rates of borrowers:")
model = CreditModel_(3)

# Print initial balance of agents
print("\nBalance of borrowers:")
for agent in model.schedule.agents:
    agent.display_balance()

# Run the model for five steps
for i in range(30):
    model.step()

    # Print the balance of each agent
    for agent in model.schedule.agents:
        if agent.loan_amount > 0:
            print(f"Borrower {agent.unique_id + 1}: Balance at step {model.step_number}: {agent.loan_amount:.2f}.- USD")
        else:
            if agent.first_zero_balance_step is None:  # Check if this is the first zero balance
                agent.first_zero_balance_step = i + 1  # Save the step number
            print(f"Borrower {agent.unique_id + 1}: Balance at step {i + 1}: 0.00.- USD")

# Print the payback day of borrowers
print("--"*20 +"\nPay back day of borrowers:")
for agent in model.schedule.agents:
    payback_day = agent.first_zero_balance_step if agent.first_zero_balance_step else "Not applicable"
    print(f"Borrower {agent.unique_id + 1}: Pay back day: {payback_day}.")


Interest rates of borrowers:
Interest rate of Borrower 1: 5.44%
Interest rate of Borrower 2: 5.68%
Interest rate of Borrower 3: 4.12%

Balance of borrowers:
Borrower 1: Balance at step 0: 100.00.- USD
Borrower 2: Balance at step 0: 100.00.- USD
Borrower 3: Balance at step 0: 100.00.- USD
Borrower 1: Balance at step 1: 94.56.- USD
Borrower 2: Balance at step 1: 94.32.- USD
Borrower 3: Balance at step 1: 95.88.- USD
Borrower 1: Balance at step 2: 89.12.- USD
Borrower 2: Balance at step 2: 88.64.- USD
Borrower 3: Balance at step 2: 91.76.- USD
Borrower 1: Balance at step 3: 83.68.- USD
Borrower 2: Balance at step 3: 82.96.- USD
Borrower 3: Balance at step 3: 87.64.- USD
Borrower 1: Balance at step 4: 78.24.- USD
Borrower 2: Balance at step 4: 77.28.- USD
Borrower 3: Balance at step 4: 83.52.- USD
Borrower 1: Balance at step 5: 72.80.- USD
Borrower 2: Balance at step 5: 71.60.- USD
Borrower 3: Balance at step 5: 79.40.- USD
Borrower 1: Balance at step 6: 67.36.- USD
Borrower 2: Balance at 

In [53]:
print(model.schedule.agents[0].display_balance())

Borrower 1: Balance at step 0: -32.00.- USD
None


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [54]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.5.0-1022-azure
Datetime: 2024-07-04 13:43:30
Python Version: 3.10.14
-----------------------------------
